# Topic Modeling (LDA)
## Setup


In [1]:
!pip install --upgrade gensim

In [2]:
!pip install nltk
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.1 MB/s eta 0:00:00


## import libraries

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import numpy as np
import json
import glob

#gensim
import gensim
import gensim.corpora as copora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Data Prep

In [5]:
def laod_data(file):
  with open (file, "r", encoding="utf-8") as f:
    data = json.load(f)
  return (data)

def write_data(file, data):
  with open (file, "w", encoding="utf-8") as f:
    json.dump(data, f, indent=4)

In [6]:
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [45]:
data = laod_data("/content/open_answers_and_comments.json")["comments"]

print(data[0][0:90])

Good to know; not sure when it was introduced, but as of at least node v0.10.33 console.lo


In [46]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
  nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
  texts_out = []
  for text in texts:
    doc = nlp(text)
    new_text = []
    for token in doc:
      if token.pos_ in allowed_postags:
        new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return(texts_out)

lemmatized_text = lemmatization(data)
print(lemmatized_text[0][0:90])

good know sure introduce least implicitly apply util.inspect argument assume 1st one forma


In [47]:
def gen_words(texts):
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return (final)

data_words = gen_words(lemmatized_text)

print(data_words[0][0:30])

['good', 'know', 'sure', 'introduce', 'least', 'implicitly', 'apply', 'util', 'inspect', 'argument', 'assume', 'st', 'one', 'format', 'string', 'happy', 'util', 'inspect', 'default', 'option', 'simply', 'need', 'require', 'util', 'same', 'accept', 'only', 'object', 'inspect', 'least']


In [48]:
id2word = gensim.corpora.Dictionary(data_words)

corpus = []

for text in data_words:
  new = id2word.doc2bow(text) # bow -> bag of words
  corpus.append(new)

print(corpus[0][0:20]) #each tuple contains index of the word along with the frequency of that word.

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 4), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1)]


In [49]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                    id2word=id2word,
                                    num_topics=10,
                                    random_state=100,
                                    update_every=1,
                                    chunksize=100,
                                    passes=10,
                                    alpha="auto")

## Visualization of Data

In [50]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus, id2word, mds="mmds", R=9)
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.426481 -0.212906       1        1  45.405887
8     -0.083456  0.390475       2        1  11.294135
9     -0.327161  0.215525       3        1  11.054280
2      0.199107 -0.332635       4        1   7.633524
6      0.372085 -0.084171       5        1   6.999565
5      0.282367  0.136368       6        1   4.689485
0      0.137943  0.263386       7        1   4.094653
1     -0.048024 -0.289343       8        1   3.588565
4     -0.159778 -0.048760       9        1   3.519945
7      0.053398 -0.037939      10        1   1.719961, topic_info=           Term        Freq       Total Category  logprob  loglift
1        answer  296.000000  296.000000  Default   9.0000   9.0000
46          use  739.000000  739.000000  Default   8.0000   8.0000
453    question  209.000000  209.000000  Default   7.0000   7.0000
889        type  203.000000  203.000000  Default   6.0000   6.0000
75           go  118.000000  118.000000  Default   5.0000   5.0000
...         ...         ...         ...      ...      ...      ...
787       above   22.957870   23.772796  Topic10  -3.2265   4.0280
1590  important   18.636027   19.450951  Topic10  -3.4351   4.0201
1479        put   12.474964   13.289765  Topic10  -3.8364   3.9996
1193      query   10.825148   11.639855  Topic10  -3.9783   3.9903
1616       node   10.198312   11.013249  Topic10  -4.0379   3.9860

[100 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
787      10  0.967492    above
91        8  0.975930  address
1         2  0.996570   answer
779       6  0.986070    array
538       4  0.978698      ask
...     ...       ...      ...
117       6  0.969208  usually
403       7  0.974104  version
289       8  0.949126  warning
59        1  0.996878     work
1021      2  0.986659    wrong

[92 rows x 3 columns], R=9, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 9, 10, 3, 7, 6, 1, 2, 5, 8])

In [51]:
print('\nPerplexity: ', lda.log_perplexity(corpus,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.901759292432699

Coherence Score:  0.44282680540734115
